In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory, RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTState, RESTStateV3
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")

INFO:rdkit:Enabling RDKit 2021.03.5 jupyter extensions


conducting subsequent work with the following platform: CUDA


# Run scaling test v2 (allows testing PME)

In [2]:
import copy


In [129]:
def test_energy_scaling():
    """
        Test whether the energy of a REST-ified system is equal to the energy of the system with terms manually scaled by
        the same factor as is used in REST.  T_min is 298 K and the thermodynamic state has temperature 600 K.
    """

    # Set temperatures
    T_min = 300.0 * unit.kelvin
    T = 600 * unit.kelvin

    ## CASE 1: alanine dipeptide in vacuum
    # Create vanilla system
    ala = AlanineDipeptideVacuum()
    system = ala.system
    system.removeForce(4)
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms)
    REST_system = factory.REST_system

    # Check energy scaling
    compare_energies(topology, REST_system, system, positions, rest_atoms, T_min, T)

    ## CASE 2: alanine dipeptide in solvent
    # Create vanilla system
    ala = AlanineDipeptideExplicit()
    system = ala.system
    system.removeForce(4)
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Check energy scaling
    compare_energies(topology, REST_system, system, positions, rest_atoms, T_min, T)

    ## CASE 3: alanine dipeptide in solvent with repartitioned hybrid system
    # Create repartitioned hybrid system for lambda 0 endstate
    atp, system_generator = generate_atp(phase='solvent')
    htf = generate_dipeptide_top_pos_sys(atp.topology,
                                         new_res='THR',
                                         system=atp.system,
                                         positions=atp.positions,
                                         system_generator=system_generator,
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)
    system = htf.hybrid_system
    system.removeForce(0) # Remove barostat

    # Create REST-ified hybrid system
    res1 = list(htf.hybrid_topology.residues)[1]
    rest_atoms = [atom.index for atom in list(res1.atoms)]
    factory = RESTTopologyFactoryV3(system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Check energy scaling
    compare_energies(htf.hybrid_topology, REST_system, system, htf.hybrid_positions, rest_atoms, T_min, T)

In [144]:
def compare_energies(topology, REST_system, other_system, positions, rest_atoms, T_min, T):
    # Remove standard NonbondedForce (which contains reciprocal space) from REST_system
    REST_system.removeForce(6)
    
# #     # Zero nb forces
# #     nb_force = REST_system.getForce(3)
# #     for i in range(nb_force.getNumParticles()):
# #         params = nb_force.getParticleParameters(i)
# #         params = list(params)
# #         params[-1] = params[-1]*0
# #         nb_force.setParticleParameters(i, params)
    
# #     nb_force = REST_system.getForce(4)
# #     for i in range(nb_force.getNumParticles()):
# #         params = nb_force.getParticleParameters(i)
# #         params = list(params)
# #         params[-1] = params[-1]*0
# #         nb_force.setParticleParameters(i, params)

#     # Zero exceptions force
#     exceptions_force = REST_system.getForce(5)
#     for i in range(exceptions_force.getNumBonds()):
#         p1, p2, params = exceptions_force.getBondParameters(i)
#         params = list(params)
#         params[-4] = params[-4]*0
#         params[-2] = params[-2]*0
# #         params[-1] = params[-1]*0
#         exceptions_force.setBondParameters(i, p1, p2, params)
    
    # Prevent reciprocal space from being used in the original system energy
    other_system.getForce(3).setReciprocalSpaceForceGroup(31)    
    
#     nb_force = other_system.getForce(3)
#     for i in range(nb_force.getNumExceptions()):
#         p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
#         nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)

#     for i in range(nb_force.getNumParticles()):
#         charge, sigma, epsilon = nb_force.getParticleParameters(i)
#         nb_force.setParticleParameters(i, charge, sigma, epsilon*0)
    
    # Create thermodynamic state
    lambda_zero_alchemical_state = RESTStateV3.from_system(REST_system)
    thermostate = ThermodynamicState(REST_system, temperature=T_min)
    compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                              composable_states=[lambda_zero_alchemical_state])
    
    # Set alchemical parameters
    beta_0 = 1 / (kB * T_min)
    beta_m = 1 / (kB * T)
    compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

    # Get energy
    integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context = compound_thermodynamic_state.create_context(integrator)
    context.setPositions(positions)
    REST_energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit_system(unit.md_unit_system)

    # Get solvent atoms
    positive_ion_name = "NA"
    negative_ion_name = "CL"
    water_name = "HOH"
    solvent_atoms = []
    if 'openmm' in topology.__module__:
        atoms = topology.atoms()
    elif 'mdtraj' in topology.__module__:
        atoms = topology.atoms
    for atom in atoms:
        if atom.residue.name == positive_ion_name or atom.residue.name == negative_ion_name or atom.residue.name == water_name:
            solvent_atoms.append(atom.index)
    
    # Compute energy for non-RESTified system
    # Determine regions and scaling factors
    nonrest_atoms = [i for i in range(other_system.getNumParticles()) if i not in rest_atoms]
    rest_scaling = beta_m / beta_0
    inter_scaling = np.sqrt(beta_m / beta_0)
    
    # Compute 3 energies: 
    ## Test case 0: rest-rest energy
    ## Test case 1: nonrest-nonrest energy
    ## Test case 2: total energy
    ## Test case 3: nonrest_solvent - rest (this will only be a nonbonded energy)
    ## Test case 4: nonrest_solvent - nonrest-solvent (this will only be a nonbonded energy)
    ## Test case 5: rest-rest nonbonded energy
    unmodified_energies = []
    for test_case in range(6):
        system_copy = copy.deepcopy(other_system)
        
        if test_case != 2:
            bond_force = system_copy.getForce(0)
            for bond_index in range(bond_force.getNumBonds()):
                p1, p2, length, k = bond_force.getBondParameters(bond_index)
                particles = [p1, p2]
                if all(x in rest_atoms for x in particles):
                    if test_case in [1, 3, 4, 5]:
                        bond_force.setBondParameters(bond_index, p1, p2, length, k*0)
                    else:
                        continue
                elif all(x in nonrest_atoms for x in particles):
                    if test_case in [0, 3, 4, 5]:
                        bond_force.setBondParameters(bond_index, p1, p2, length, k*0)
                    else:
                        continue
                else:
                    bond_force.setBondParameters(bond_index, p1, p2, length, k*0)

            angle_force = system_copy.getForce(1)
            for angle_index in range(angle_force.getNumAngles()):
                p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
                particles = [p1, p2, p3]
                if all(x in rest_atoms for x in particles):
                    if test_case in [1, 3, 4, 5]:
                        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*0) 
                    else:
                        continue
                elif all(x in nonrest_atoms for x in particles):
                    if test_case in [0, 3, 4, 5]:
                        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*0) 
                    else:
                        continue
                else:
                    angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k*0) 

            torsion_force = system_copy.getForce(2)
            for torsion_index in range(torsion_force.getNumTorsions()):
                p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
                particles = [p1, p2, p3, p4]
                if all(x in rest_atoms for x in particles):
                    if test_case in [1, 3, 4, 5]:
                        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*0)
                    else:
                        continue
                elif all(x in nonrest_atoms for x in particles):
                    if test_case in [0, 3, 4, 5]:
                        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*0)
                    else:
                        continue
                else:
                    torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k*0)

            nb_force = system_copy.getForce(3)
            for i in range(nb_force.getNumExceptions()):
                p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(i)
                particles = [p1, p2]
                if all(x in rest_atoms for x in particles):
                    if test_case in [1, 4]:
                        nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)
                    else:
                        continue
                elif all(x in nonrest_atoms for x in particles):
                    if test_case in [0, 5]:
                        nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)
                    elif test_case in [3, 4]:
                        if all(x in solvent_atoms for x in particles):
                            continue
                        else:
                            nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)
                    else:
                        continue
                else:
                    if test_case == 3:
                        if any(x in solvent_atoms for x in particles):
                            continue
                        else:
                            nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)
                    else:
                        nb_force.setExceptionParameters(i, p1, p2, chargeProd*0, sigma, epsilon*0)

            for i in range(nb_force.getNumParticles()):
                charge, sigma, epsilon = nb_force.getParticleParameters(i)
                if i in rest_atoms:
                    if test_case in [1, 4]:
                        nb_force.setParticleParameters(i, charge*0, sigma, epsilon*0)
                    else:
                        continue
                else:
                    if test_case in [0, 5]:
                        nb_force.setParticleParameters(i, charge*0, sigma, epsilon*0)
                        
                    elif test_case in [3, 4]:
                        if i in solvent_atoms:
                            continue
                        else:
                            nb_force.setParticleParameters(i, charge*0, sigma, epsilon*0)
                            
                    else:
                        continue     
        
        
        # Get energy
        thermostate = ThermodynamicState(system_copy, temperature=T_min)
        integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context = thermostate.create_context(integrator)
        context.setPositions(positions)
        system_copy_energy = context.getState(getEnergy=True, groups=set([0])).getPotentialEnergy().value_in_unit_system(unit.md_unit_system)
        unmodified_energies.append(system_copy_energy)
    
    print(unmodified_energies)
    nonrest_solvent_rest = unmodified_energies[3] - unmodified_energies[5] - unmodified_energies[4]
    nonrest_protein_rest = unmodified_energies[2] - unmodified_energies[0] - unmodified_energies[1] - nonrest_solvent_rest
    unmodified_energy = unmodified_energies[0] * rest_scaling + unmodified_energies[1] +  nonrest_solvent_rest * rest_scaling + nonrest_protein_rest * inter_scaling
    print(unmodified_energy)
    print(REST_energy)
#     assert np.isclose(REST_energy, unmodified_energy), f"REST energy was {REST_energy} and unmodified_energy was {unmodified_energy}"
                            

In [145]:
test_energy_scaling()


INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 0
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


[99.16266923641906, -43.868905857090624, -88.0885568982415, 97.89623674058818, 0.0, 97.89623674058818]
-95.67418220941221
-95.67417528490705
[343.0211989972927, 176160.56311972506, 176331.73873220527, 176250.82582921648, 175941.74369224504, 341.7547697201143]
176217.32726348194
176217.32516327008


DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 16, 19, 13, 17, 15]
INFO:geometry:omitted_bonds: []
INFO:ge

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.67298667212561
INFO:geometry:final reduced energy 29.78946596595155
INFO:geometry:sum of energies: 29.789467813837646
INFO:geometry:magnitude of difference in the energies: 1.8478860965842614e-06
INFO:geometry:Final logp_proposal: -27061.38503748967


added energy components: [('CustomBondForce', 0.0005202039273265048), ('CustomAngleForce', 0.45111977562481365), ('CustomTorsionForce', 7.25046332853325), ('CustomBondForce', 12.970883364040219)]


INFO:relative:*** Generating RepartitionedHybridTopologyFactory ***
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:rel

[135.49484040973738, 120113.38875204608, 120160.83303260534, 119966.75123901477, 119905.25605392292, 84.50364245017815]
120123.64023183695
120123.63899385846


# Check whether waters have valence energy

In [72]:
import openmmtools
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTStateV3
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
# REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("Reference")

In [70]:
waterbox = openmmtools.testsystems.PMEWaterBox()

In [71]:
[system, positions] = [waterbox.system, waterbox.positions]

In [74]:
platform = configure_platform(platform_name='CUDA')
print(platform.getName())
print(platform.getPropertyDefaultValue('CudaPrecision'))
print(platform.getPropertyDefaultValue('DeterministicForces'))

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other, platform=platform)
context_other.setPositions(positions)

# Get energy components for non-rest system
components_other = [component for component in compute_potential_components(context_other, beta=beta, platform=platform)]


conducting subsequent work with the following platform: CUDA
CUDA
mixed
true
conducting subsequent work with the following platform: CUDA


In [75]:
components_other

[('HarmonicBondForce', 0.0),
 ('HarmonicAngleForce', 0.0),
 ('NonbondedForce', -644.5640399939954),
 ('AndersenThermostat', 0.0)]

This shows that waters don't contribute to the valence energy